In [ ]:
%%capture
!pip install transformers
!pip install gradio==4.36.0
!pip install pytube
!pip install openai==0.28
!pip install tiktoken
!pip install datasets
!pip install langchain
!pip install sentence-transformers
!pip install chromadb
!pip install langchain-community
!pip install torch

In [ ]:
import transformers
print(transformers.__version__)


4.46.2


In [ ]:
import torch
import openai
import gradio as gr
import pytube as pt
import soundfile as sf

from io import BytesIO
from transformers import pipeline
from huggingface_hub import model_info
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma

In [ ]:
openai.api_key = ""
llm = ChatOpenAI(model_name="gpt-4-turbo", temperature=0, openai_api_key=openai.api_key)
restart_sequence = "\n"

<ipython-input-4-fd20d45149c2>:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-4-turbo", temperature=0, openai_api_key=openai.api_key)


In [ ]:
llm = ChatOpenAI(model_name="gpt-4-turbo", temperature=0, openai_api_key=openai.api_key)
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/LaBSE')
synthesiser = pipeline("text-to-speech", model='facebook/mms-tts-yor')

MODEL_NAME = "omoekan/whisper-small-yoruba" #this always needs to stay in line 8 :D sorry for the hackiness
lang = "yo"

device = 0 if torch.cuda.is_available() else "cpu"
pipe = pipeline(
    task="automatic-speech-recognition",
    model=MODEL_NAME,
    chunk_length_s=30,
    device=device,
)

pipe.model.config.forced_decoder_ids = pipe.tokenizer.get_decoder_prompt_ids(language=lang, task="transcribe")

<ipython-input-5-23344a49eb43>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/LaBSE')
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  w

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.22k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.11k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.06k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

In [ ]:
loader = TextLoader('/content/filtered_yoruba.txt', encoding = "utf-8")
documents = loader.load()

persist_directory = '/content/chroma/'

text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=50,separators = ".")
docs = text_splitter.split_documents(documents)

#db = Chroma.from_documents(docs, embeddings)
#retriever = db.as_retriever()

embedding = OpenAIEmbeddings(openai_api_key=openai.api_key)
db = Chroma.from_documents(docs, embedding)

<ipython-input-7-dc5418bc0279>:12: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  embedding = OpenAIEmbeddings(openai_api_key=openai.api_key)


In [ ]:
len(db)

1

In [ ]:
# Build prompt
# Define prompt
question = "Ipinlẹ melo wo ni Yakubu Gowon kọkọ da ni ọdun 1967 ?"
template = """ Use the following pieces of context to answer the question at the end in yoruba. If you don't know the answer, just say rara o, don't try to make up an answer.Keep the answer as concise as possible. Please don't make anything up, just reply with
rara o if you are not at least 90% sure of your answer.
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

#Define the llm to use
llm = ChatOpenAI(model_name="gpt-4-turbo", temperature=0, openai_api_key=openai.api_key)
# Run chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=db.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

result = qa_chain({"query": question})
final_result = result['result']

print(final_result)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(


méjìlá


In [ ]:
result['source_documents'][1]

Document(metadata={'source': '/content/filtered_yoruba.txt'}, page_content='.86\xa0\']\n\nQuestion: Ọwọ orilẹ-ede wo ni Sierra Leone ti gba ominiria?\nAnswer: [\'Ilẹ̀ Gẹ̀ẹ́sì\']\n\nQuestion: Iwe melo ni Thomas Carlyle ti kọ?\nAnswer: [\'Oogun\']\n\nQuestion: Ọdun wo ni a sọ Femi Falana di amofin agba?\nAnswer: [\'Ọdun 2012\']\n\nQuestion: Ki lorukọ ọba ookanlelogun ilu Warri ?\nAnswer: [\'Ogiame Atuwatse III\']\n\nQuestion: Iku wo lo pa Adolph Hitler ?\nAnswer: [\'suicide\']\n\nQuestion: Ẹgbẹ oṣelu wo ni o se iṣakoso Ikọle ekiti ni ọdun 2019 ?\nAnswer: [\'APC\']\n\nQuestion: Elo ni barẹli epo ti wọn reti pe ilẹ-ise ifọ epo Dangote yoo ma seto lojumọ?\nAnswer: [\'650,000\']\n\nQuestion: Ta ni onitọhun ti o sọ Naijiria ni orukọ ti o n jẹ ?\nAnswer: [\'Flora Louise Shaw\']\n\nQuestion: Ọjo melo ni Fisayo Soyombo lo ninu ewon Ikọyi nigbati o n se se iwadi?\nAnswer: [\'ọjọ́ márùn-ún\']\n\nQuestion: ọdun wo ni Chinyere Yvonne Okoro bẹrẹ isẹ oṣere?\nAnswer: [\'2002\']\n\nQuestion: Ni ọdu

In [ ]:
get_answer("Awọn ipinlẹ melo lo wa ni Naijiria?")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


'mẹ́tàdínlógójì'

In [ ]:
def transcribe(microphone, file_upload):
    warn_output = ""
    if (microphone is not None) and (file_upload is not None):
        warn_output = (
            "WARNING: You've uploaded an audio file and used the microphone. "
            "The recorded file from the microphone will be used and the uploaded audio will be discarded.\n"
        )

    elif (microphone is None) and (file_upload is None):
        return "ERROR: You have to either use the microphone or upload an audio file"

    file = microphone if microphone is not None else file_upload

    text = pipe(file)["text"]

    return warn_output + text

In [ ]:
def get_prompt(microphone, file_upload):
  question = transcribe(microphone, file_upload)
  prompt = """ Use the following pieces of context to answer the question at the end in yoruba. If you don't know the answer, just say rara o, don't try to make up an answer.Keep the answer as concise as possible. Please don't make anything up, just reply with
  rara o if you are not at least 90% sure of your answer.
  {context}
  Question: {question}
  Helpful Answer:"""
  return question, prompt


In [ ]:
# Build prompt
def get_answer(microphone, file_upload):
  question, template = get_prompt(microphone, file_upload)
  # Define prompt
  QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

  #Define the llm to use
  llm = ChatOpenAI(model_name="gpt-4-turbo", temperature=0, openai_api_key=openai.api_key)
  # Run chain
  qa_chain = RetrievalQA.from_chain_type(
      llm,
      retriever=db.as_retriever(),
      return_source_documents=False,
      chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
  )

  result = qa_chain({"query": question})
  final_result = result['result']

  return final_result

In [ ]:
from transformers import VitsModel, AutoTokenizer
import torch
import numpy as np


def synthesis(microphone, file_upload):
  question = transcribe(microphone, file_upload)
  text = get_answer(microphone, file_upload)
  tts_output = synthesiser(text)
  audio_data = tts_output["audio"][0]

  with BytesIO() as buffer:
      sf.write(buffer, audio_data, 16000, format='wav')
      audio_bytes = buffer.getvalue()

  return question, text, audio_bytes


In [ ]:
def synthesis(text):
  text = text
  tts_output = synthesiser(text)
  audio_data = tts_output["audio"][0]

  with BytesIO() as buffer:
      sf.write(buffer, audio_data, 16000, format='wav')
      audio_bytes = buffer.getvalue()

  return audio_bytes


css = """
body {
    background-color: white !important;
}
"""


demo = gr.Interface(
    fn=synthesis,
    inputs=[
        gr.Textbox(label="Input text"),
    ],
    outputs= [gr.Audio(label="Generated Speech")],
    title="YorubaAI",
    description=(
        "Chat with YourbaAI and get answer to your questions in Yoruba"
    ),
    allow_flagging="never",
    theme="light",
    css=css,
)



if __name__ == "__main__":
  demo.launch(debug=True)

/usr/local/lib/python3.10/dist-packages/gradio/blocks.py:983: UserWarning: Cannot load light. Caught Exception: module 'huggingface_hub.utils' has no attribute '_errors'
  warnings.warn(f"Cannot load {theme}. Caught Exception: {str(e)}")


IMPORTANT: You are using gradio version 4.36.0, however version 4.44.1 is available, please upgrade.
--------
Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://20f56305ad8ad63810.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://20f56305ad8ad63810.gradio.live


In [ ]:
demo = gr.Interface(
    fn=synthesis,
    inputs=[
        gr.Audio(sources="microphone", type="filepath"),
        gr.Audio(sources="upload", type="filepath"),
    ],
    outputs= [gr.Textbox(label="Question asked"), gr.Textbox(label="Generated text"), gr.Audio(label="Generated Speech")],
    title="YorubaAI",
    theme="huggingface",
    description=(
        "Chat with YourbaAI and get answer to your questions in Yoruba"
    ),
    allow_flagging="never",
)


if __name__ == "__main__":
  demo.launch(debug=True)

/usr/local/lib/python3.10/dist-packages/gradio/blocks.py:983: UserWarning: Cannot load huggingface. Caught Exception: 404 Client Error: Not Found for url: https://huggingface.co/api/spaces/huggingface (Request ID: Root=1-66d95f4a-6fce835768a8240f103910f8;24bf9318-556b-420b-92c1-6639cbf86a10)

Sorry, we can't find the page you are looking for.
  warnings.warn(f"Cannot load {theme}. Caught Exception: {str(e)}")


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://bfda19f73754be9095.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:496: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gradio/queueing.py", line 532, in process_events
    response = await route_utils.call_process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/route_utils.py", line 276, in call_process_api
    output = await app.get_blocks().process_api(
 

In [ ]:
from datasets import load_dataset


ds = load_dataset("masakhane/afriqa", "yor")

Generating train split:   0%|          | 0/360 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/361 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/332 [00:00<?, ? examples/s]

In [ ]:
ds

DatasetDict({
    train: Dataset({
        features: ['question', 'answers', 'lang', 'split', 'translated_question', 'translated_answer', 'translation_type'],
        num_rows: 360
    })
    validation: Dataset({
        features: ['question', 'answers', 'lang', 'split', 'translated_question', 'translated_answer', 'translation_type'],
        num_rows: 361
    })
    test: Dataset({
        features: ['question', 'answers', 'lang', 'split', 'translated_question', 'translated_answer', 'translation_type'],
        num_rows: 332
    })
})

In [ ]:
import pandas as pd
ds = pd.read_excel("/content/drive/MyDrive/Otherweb/yoruba.xlsx")

In [ ]:
# Writing text to a file
with open('yoruba.txt', 'w') as file:
  for i in range(330):
    file.write(f"Question: {ds['train']['question'][i]}\n")
    file.write(f"Answer: {ds['train']['answers'][i]}\n")
    file.write("\n")

    """

    file.write(f"Question: {ds['test']['question'][i]}\n")
    file.write(f"Answer: {ds['test']['answers'][i]}\n")
    file.write("\n")

    file.write(f"Question: {ds['validation']['question'][i]}\n")
    file.write(f"Answer: {ds['validation']['answers'][i]}\n")
    file.write("\n")
    """

In [ ]:
ds.set_format("pandas")

In [ ]:
data = ds["train"][:]

In [ ]:
data.head()

,id,url,title,text
0,598,https://yo.wikipedia.org/wiki/A,A,el: άλφα\n yo: a (ah)\n\naa ab ac ad ae af ag ...
1,599,https://yo.wikipedia.org/wiki/B,B,B\n\n el: βήτα\n\nba bb bc bd be bf bg bh bi b...
2,600,https://yo.wikipedia.org/wiki/C,C,C\n\nca cb cc cd ce cf cg ch ci cj ck cl cm cn...
3,601,https://yo.wikipedia.org/wiki/D,D,D\n\n ar: ?\n el: δέλτα (Δ δ)\n he: ?\n hi: ?\...
4,602,https://yo.wikipedia.org/wiki/E,E,E\n\n ar: ?\n Cyrillic: ?\n el: έψιλον (Ε ε)\n...


In [ ]:
data.to_csv("/content/drive/MyDrive/Otherweb/yoruba.xlsx", index=False)

In [ ]:
# prompt: read the test file yoruba.txt

with open('/content/filtered_yoruba.txt', 'r') as file:
  content = file.read()
print(content)


Question: Ki ni iṣẹ ti Chief Bọde Thomas n ṣe ki o to kuro laye ?
Answer: ['alakoso ile ijọsin Afirika ti Naijiria']

Question: Awọn ipinlẹ melo lo wa ni Naijiria?
Answer: mẹ́rìndínlógójì

Question: Ipinlẹ melo wo ni Yakubu Gowon kọkọ da ni ọdun 1967 ?
Answer: méjìlá

Question: Ijọba ibilẹ meloo lo wa ni orilẹ ede Naijiria?
Answer: ẹ̀rin-dín-ní-ojì dín ní ẹgbẹ́rin

Question: Ta ni o tumọ bibeli si ede Yoruba ?
Answer: ['Samuel Ajayi Crowther']

Question: Ni ọdun wo ni a da ile-ẹkọ Rufus Giwa Polytechnic silẹ?
Answer: ['Ọdun 1979']

Question: Ọdun wo s'ọdun wo ni Olúṣẹ́gun Ọ̀ṣọbà jẹ  Gómìnà Ìpínlẹ̀ Ògùn n'ilẹ Naijiria?
Answer: ['oṣu kini ọdun 1992 titi di oṣu kọkanla ọdun 1993']


In [ ]:
def remove_empty_answers_from_text_file(file_path):
    # Read data from the text file
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    # Initialize variables
    filtered_data = []
    question = None
    answer = None

    # Process each line
    for line in lines:
        if line.startswith("Question:"):
            question = line.strip()
        elif line.startswith("Answer:"):
            answer = line.strip()
            # Check if the answer is empty (e.g., 'Answer: []')
            if answer != "Answer: []":
                filtered_data.append((question, answer))

    # Write the filtered data back to a new file
    with open('filtered_yoruba.txt', 'w', encoding='utf-8') as file:
        for q, a in filtered_data:
            file.write(f"{q}\n{a}\n\n")

# Usage example
file_path = '/content/yoruba.txt'  # Replace with your file path
remove_empty_answers_from_text_file(file_path)


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("lelapa/InkubaLM-0.4B", trust_remote_code=True, use_auth_token="")
tokenizer = AutoTokenizer.from_pretrained("lelapa/InkubaLM-0.4B", trust_remote_code=True, use_auth_token="")

model.to('cuda')


/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:469: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vulavulaslm.py:   0%|          | 0.00/42.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/lelapa/InkubaLM-0.4B:
- vulavulaslm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


pytorch_model.bin:   0%|          | 0.00/2.66G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/960 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/991k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.95M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

VulavulaLlamaForCausalLM(
  (model): VulavulaLlamaModel(
    (embed_tokens): Embedding(61788, 2048)
    (layers): ModuleList(
      (0-7): 8 x VulavulaLlamaDecoderLayer(
        (self_attn): VulavulaLlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (v_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): VulavulaLlamaRotaryEmbedding()
        )
        (mlp): VulavulaLlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (down_proj): Linear(in_features=5632, out_features=2048, bias=False)
          (up_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): VulavulaLlamaRMSNorm()
        (post_attention_layernorm): VulavulaLlamaRMSNorm()
  